<a href="https://colab.research.google.com/github/Swapn2/PyTorch/blob/main/hyperparameter_tunning_bayesian.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install optuna

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 395.9/395.9 kB 14.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 247.0/247.0 kB 13.1 MB/s eta 0:00:00


In [2]:
import optuna
from sklearn.datasets import load_diabetes
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [3]:
import pandas as pd

In [4]:
# Load the Pima Indian Diabetes dataset (from UCI repository)
url = "https://raw.githubusercontent.com/jbrownlee/Datasets/master/pima-indians-diabetes.data.csv"
columns = ['Pregnancies', 'Glucose', 'BloodPressure', 'SkinThickness', 'Insulin', 'BMI',
           'DiabetesPedigreeFunction', 'Age', 'Outcome']

# Load the dataset
df = pd.read_csv(url, names=columns)

df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [5]:
import numpy as np
cols_with_missing_vals = ['Glucose','BloodPressure','SkinThickness','Insulin','BMI']
df[cols_with_missing_vals] = df[cols_with_missing_vals].replace(0,np.nan)
df.fillna(df.mean(),inplace=True)

print(df.isnull().sum())

Pregnancies                 0
Glucose                     0
BloodPressure               0
SkinThickness               0
Insulin                     0
BMI                         0
DiabetesPedigreeFunction    0
Age                         0
Outcome                     0
dtype: int64


In [6]:
x= df.iloc[:,:-1]
y= df.iloc[:,-1]

In [7]:
x

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age
0,6,148.0,72.0,35.00000,155.548223,33.6,0.627,50
1,1,85.0,66.0,29.00000,155.548223,26.6,0.351,31
2,8,183.0,64.0,29.15342,155.548223,23.3,0.672,32
3,1,89.0,66.0,23.00000,94.000000,28.1,0.167,21
4,0,137.0,40.0,35.00000,168.000000,43.1,2.288,33
...,...,...,...,...,...,...,...,...
763,10,101.0,76.0,48.00000,180.000000,32.9,0.171,63
764,2,122.0,70.0,27.00000,155.548223,36.8,0.340,27
765,5,121.0,72.0,23.00000,112.000000,26.2,0.245,30
766,1,126.0,60.0,29.15342,155.548223,30.1,0.349,47


In [8]:
y

,Outcome
0,1
1,0
2,1
3,0
4,1
...,...
763,0
764,0
765,0
766,1


In [13]:
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size =.3, random_state= 42)

In [14]:
scaler = StandardScaler()
x_train = scaler.fit_transform(x_train)
x_test = scaler.transform(x_test)

In [15]:
x_train.shape

(537, 8)

In [16]:
x_test.shape

(231, 8)

In [17]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score

In [18]:
def objective(trial):
  #  sugesting the values of the hyperparameters search space
  n_estimators = trial.suggest_int('n_estimators', 50 ,200)
  max_depth = trial.suggest_int('max_depth' ,3,20)

  #  creating the randomforestclassifier with suggested hyperparameters
  model = RandomForestClassifier(
      n_estimators= n_estimators,
      max_depth= max_depth,
      random_state=42
  )
  score = cross_val_score(model , x_train , y_train,cv = 3 , scoring= 'accuracy').mean()
  return score



In [24]:
study = optuna.create_study(direction = 'maximize', sampler = optuna.samplers.TPESampler())
study.optimize(objective, n_trials= 50)

[I 2025-07-19 05:31:21,163] A new study created in memory with name: no-name-fb8bc5a7-8016-4eca-b237-89a5b2b518df
[I 2025-07-19 05:31:22,095] Trial 0 finished with value: 0.7746741154562384 and parameters: {'n_estimators': 150, 'max_depth': 18}. Best is trial 0 with value: 0.7746741154562384.
[I 2025-07-19 05:31:23,039] Trial 1 finished with value: 0.7728119180633147 and parameters: {'n_estimators': 120, 'max_depth': 14}. Best is trial 0 with value: 0.7746741154562384.
[I 2025-07-19 05:31:23,492] Trial 2 finished with value: 0.7728119180633147 and parameters: {'n_estimators': 54, 'max_depth': 15}. Best is trial 0 with value: 0.7746741154562384.
[I 2025-07-19 05:31:24,701] Trial 3 finished with value: 0.7746741154562384 and parameters: {'n_estimators': 193, 'max_depth': 12}. Best is trial 0 with value: 0.7746741154562384.
[I 2025-07-19 05:31:25,342] Trial 4 finished with value: 0.7783985102420856 and parameters: {'n_estimators': 128, 'max_depth': 13}. Best is trial 4 with value: 0.77839